### Clean Training Data
#### Inputs
- Flow data from Sensirion data gathering software (csv)
- Sensor data from Read_Write_Sensor python script

#### Outputs
- Export Flow csv file
- Export SensorData csv file

#### Notes
- Make sure to line up timescales prior to exporting
- Next step is to scale in Matlab

In [133]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime

df = pd.read_csv('sensor_walk_2.csv')
flow = pd.read_csv('flow_walk_2.csv')

In [134]:
hours = []
minutes = []
seconds = []

for i in range(0, len(flow)):
    t = flow['Time'][i]
    a = t.split(':')
    hours.append(float(a[0]))
    seconds.append(float(a[2]))
    minutes.append(float(a[1]))
    
flow['Hours'] = hours
flow['Minutes'] = minutes
flow['Seconds'] = seconds
flow.drop('Date', inplace = True, axis = 1)
flow.drop('Sample #', inplace = True, axis = 1)
flow.drop('Time', inplace = True, axis = 1)
flow.columns = ['Flow', 'hours', 'minutes', 'seconds']
flow.head()

,Flow,hours,minutes,seconds
0,-0.967,21.0,31.0,38.6283
1,-0.933,21.0,31.0,38.6287
2,-0.967,21.0,31.0,38.6292
3,-0.967,21.0,31.0,38.6297
4,-0.967,21.0,31.0,38.6302


In [135]:
seconds = []
minutes = []
hours = []

for i in range(0, len(df)):
    t = datetime.datetime.fromtimestamp(df['timeTracker'][i]).strftime('%Y-%m-%d %H:%M:%S.%f')
    a = t.split(' ')
    t = a[1].split(':')
    hours.append(float(t[0]))
    seconds.append(float(t[2]))
    minutes.append(float(t[1]))
    
df.drop('timeTracker', inplace = True, axis = 1)
df['Time Hours'] = hours
df['Time Minutes'] = minutes
df['Time Seconds'] = seconds
df.columns = ['1', '2', '3', '4', 'd1', 'd2', 'd3', 'd4', 'hours', 'minutes', 'seconds']
df.head()

,1,2,3,4,d1,d2,d3,d4,hours,minutes,seconds
0,0.448956,0.512739,0.420477,0.464948,0.000000,0.000000,0.000000,0.000000,21.0,31.0,47.392183
1,0.446636,0.510616,0.420477,0.462887,-0.126608,-0.115855,0.000000,-0.112511,21.0,31.0,47.401346
2,0.441205,0.502809,0.415176,0.455306,-0.231603,-0.290663,-0.128365,-0.281626,21.0,31.0,47.415113
3,0.434933,0.496681,0.411488,0.451459,-0.345977,-0.388910,-0.195315,-0.306400,21.0,31.0,47.426129
4,0.429211,0.494731,0.409263,0.449147,-0.402309,-0.320287,-0.197527,-0.279396,21.0,31.0,47.437084


In [136]:
df[0:10000]

,1,2,3,4,d1,d2,d3,d4,hours,minutes,seconds
0,0.448956,0.512739,0.420477,0.464948,0.000000,0.000000,0.000000,0.000000,21.0,31.0,47.392183
1,0.446636,0.510616,0.420477,0.462887,-0.126608,-0.115855,0.000000,-0.112511,21.0,31.0,47.401346
2,0.441205,0.502809,0.415176,0.455306,-0.231603,-0.290663,-0.128365,-0.281626,21.0,31.0,47.415113
3,0.434933,0.496681,0.411488,0.451459,-0.345977,-0.388910,-0.195315,-0.306400,21.0,31.0,47.426129
4,0.429211,0.494731,0.409263,0.449147,-0.402309,-0.320287,-0.197527,-0.279396,21.0,31.0,47.437084
...,...,...,...,...,...,...,...,...,...,...,...
9594,0.305152,0.292661,0.376003,0.291075,-0.009862,-0.009925,-0.007995,-0.010797,21.0,33.0,25.498945
9595,0.305066,0.292437,0.375940,0.291044,-0.009838,-0.010066,-0.007973,-0.010700,21.0,33.0,25.509655
9596,0.305024,0.292213,0.375878,0.291013,-0.009763,-0.010198,-0.007950,-0.010604,21.0,33.0,25.520670
9597,0.304898,0.291989,0.375816,0.290982,-0.009781,-0.010324,-0.007925,-0.010508,21.0,33.0,25.531870


In [166]:
flow[17076:201648]

,Flow,hours,minutes,seconds
17076,15.700,21.0,31.0,47.3921
17077,15.767,21.0,31.0,47.3927
17078,15.800,21.0,31.0,47.3933
17079,15.800,21.0,31.0,47.3939
17080,15.867,21.0,31.0,47.3945
...,...,...,...,...
201643,26.200,21.0,33.0,25.5411
201644,26.200,21.0,33.0,25.5416
201645,26.200,21.0,33.0,25.5421
201646,26.200,21.0,33.0,25.5426


In [167]:
flow = flow[17076:201648]

In [168]:
flow.head()

,Flow,hours,minutes,seconds
17076,15.700,21.0,31.0,47.3921
17077,15.767,21.0,31.0,47.3927
17078,15.800,21.0,31.0,47.3933
17079,15.800,21.0,31.0,47.3939
17080,15.867,21.0,31.0,47.3945


In [169]:
from scipy import signal
raw_data = flow['Flow']

In [170]:
flow_resample = signal.resample(raw_data, len(df))

In [171]:
len(flow_resample)

9599

In [172]:
len(df)

9599

In [173]:
export_csv = pd.DataFrame(flow_resample).to_csv(r'/Users/mikefurr/Documents/GitHub/coral_host/Jupyter_Notebooks/flow_walk_2_clean.csv', header = True, index = None)

In [174]:
export_csv = df.to_csv(r'/Users/mikefurr/Documents/GitHub/coral_host/Jupyter_Notebooks/sensor_walk_2_clean.csv', header = True, index = None)